In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from ltn_imp.automation.knowledge_base import KnowledgeBase

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

In [4]:
from utils.custom_layers import *
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
import utils.models as my_models
from utils.losses import weighted_binary_crossentropy_per_output
from utils.symmetric_net_utils import *
from utils.torch_model_manager import *

In [5]:
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, recall_score, balanced_accuracy_score, f1_score
from sklearn.model_selection import KFold
from sklearn.utils import class_weight

In [6]:
from psyki.logic import Theory
from psyki.logic.prolog import TuProlog
from psyki.ski import Injector

In [7]:
np.set_printoptions(precision=3, suppress=True)
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [8]:
X = pd.read_csv('./data/pima_indians_imputed.csv').astype(np.float32)
# X["DiabetesPedigreeFunction"] =  X["DiabetesPedigreeFunction"] * 100
# X["Pregnancies"] = X["Pregnancies"] * 10

dataset = X.copy()
# X = pd.read_csv('./data/diabetes.csv')
# y = X["Outcome"]
# X.drop(columns=["Outcome"],inplace=True)

y = X["Outcome"]
X.drop(columns=["Unnamed: 0","Outcome"],inplace=True)

In [9]:
zeros = sum(y==0)
ones = sum(y==1)

print("Class 0: ", zeros)
print("Class 1: ", ones)

unbalancement_index = abs(zeros - ones) / y.shape[0]
print(f"Unbalancement: {unbalancement_index}%")


Class 0:  500
Class 1:  268
Unbalancement: 0.3020833333333333%


In [10]:
#Normalization
scaler = MinMaxScaler()
X[X.columns] = scaler.fit_transform(X)

# Saving maxes
maxes = scaler.data_max_

In [11]:
# #TSNE to plot data
# tsne = TSNE(n_components=3, random_state=42)
# X_tsne = tsne.fit_transform(X)

# fig = plt.figure(figsize=(8, 6))
# ax = fig.add_subplot(111, projection='3d')

# # Scatter plot 3D
# ax.scatter(X_tsne[:, 0], X_tsne[:, 1], X_tsne[:, 2], c=y, marker='o')

# # Etichette degli assi
# ax.set_xlabel('Asse X')
# ax.set_ylabel('Asse Y')
# ax.set_zlabel('Asse Z')

# plt.grid(True)
# plt.show()

In [12]:
# Null count
null_counts_per_row = X.isnull().sum(axis=0)

null_counts_per_row

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

In [13]:
cw = class_weight.compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)
class_0_weight = cw[0]
class_1_weight = cw[1]

In [ ]:
layers_number = 2
units = 12

input_shape = (X.shape[1:])
EPOCHS = 100

n_splits = 4

tests_info = { # name : [learning_rate]
    "Uneducated":[0.001],
    "KBANN":[0.0001],
    "KINS":[0.01],
    "LTN":[0.001],
    "Symmetric":[0.001]
}

In [15]:
knowledge = TuProlog.from_file("./knowledge/diabetes.pl")
theory = Theory(knowledge, dataset)
for rule in theory.formulae:
    print(f"{rule.rhs} -> {rule.lhs.args.last}")

Glucose > 126.0, BMI > 30.0 -> 1.0
Glucose =< 100.0, BMI =< 25.0 -> 0.0


In [16]:
# This map will contain performances foreach tested model
test_performances = {}

# KFold for cross validation
kf = KFold(
    n_splits=n_splits,
    # random_state=42
)

# Iterating test infos
for current_test_name, current_test_params  in tests_info.items():   

    all_score = []


    specificity_knowledge = 0
    recall_knowledge = 0

    sp_kn_count = 0
    re_kn_count = 0

    # Cross validation
    for train_index, val_index in kf.split(X):
        early_stopping = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)
        
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=current_test_params[0],
        )
        
        # Subdividing dataset in training and validation fold
        train = dataset.iloc[train_index,:]
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        
        # Computing class weight
        cw = class_weight.compute_class_weight(class_weight="balanced", classes=np.unique(y_val), y=y_val)
        class_0_weight = cw[0]
        class_1_weight = cw[1]

        # Initializing base model
        uneducated = my_models.getBaseModel(
            input_shape,
            units,
            layers_number,
        )
        
        sample_weight = ((y_train == 1) * class_1_weight) + ((y_train == 0) * class_0_weight)

        if current_test_name != "LTN":
            # Tensorflow based algorithms

            output_weights = [1]

            if current_test_name == "Uneducated" or current_test_name == "Symmetric":
                model = uneducated                
            else:
                theory = Theory(knowledge, train)
                theory.set_all_formulae_trainable()

                if current_test_name == "KBANN":
                    injector = Injector.kbann(uneducated)

                    # Adapting target to KBANN structure
                    output_weights = [1, 1]
                    y_train = np.column_stack([y_train.copy(), 1-y_train])
                    y_val = np.column_stack([y_val.copy(), 1-y_val])

                elif current_test_name == "KINS":
                    injector = Injector.kins(uneducated)

                # Injecting knowledge
                model = injector.inject(theory)

            # plot_model(model, to_file='educated_structure.png', show_shapes=True, show_layer_names=True)
            # 1/
            
            epochs = EPOCHS

            if current_test_name == "Symmetric":
                epochs = epochs // 2

            #####################################
            # Compile and train selected model
            model.compile(
                optimizer = optimizer,
                loss=weighted_binary_crossentropy_per_output(output_weights),
                metrics=['accuracy'],            
            )

            model.fit(
                X_train,
                y_train,
                sample_weight=sample_weight,
                batch_size=32,
                epochs=epochs,
                callbacks=[early_stopping]
            )
            #####################################

            if current_test_name == "Symmetric":
                output_weights = [class_0_weight,class_1_weight*1.6,3.8,0.9]

                # Initializing symmetric model
                symmetric = my_models.getQuadModel(
                    input_shape,
                    model,                    
                )       

                # Creating mask and preparing data for symmetric model training
                mask = get_mask(X_train,y_train, maxes)            
                x_symmetric, y_symmetric, sample_weight = create_4_tuple(X_train, y_train, mask)

                #################################
                #Compiling and training symmetric model

                symmetric.compile(
                    optimizer = optimizer,  
                    loss=weighted_binary_crossentropy_per_output(output_weights), 
                    metrics=['accuracy']
                )

                symmetric.fit(
                    x_symmetric,
                    y_symmetric,
                    sample_weight=sample_weight,
                    batch_size=32,
                    epochs=EPOCHS//2,
                    # class_weight={0:[cw[0],1,1],1:[cw[1],1,1]},
                    callbacks=[early_stopping]
                )
                ###############################

            # Prediction on validation set    
            prediction = model.predict(X_val)

            if current_test_name == "KBANN":
                # Transforming y to single value target
                y_val = y_val[:,0]
                prediction = 1-prediction[:,1]
        else:
            #LTN
            # caching datasets
            X_train_tosave = X_train.copy()
            X_val_tosave = X_val.copy()

            X_train_tosave["Outcome"] = y_train
            X_val_tosave["Outcome"] = y_val

            cache_datasets({
                "train": X_train_tosave,
                "test":X_val_tosave,
                "val":X_val_tosave
            })

            # Creating LTN model
            kb = KnowledgeBase("./knowledge/config.yaml")
            model = kb.predicates["Diabetic"]

            train_model(model, kb, lr=current_test_params[0])

            prediction = predict(model, X_val.to_numpy())


        # Preparing data to evaluate model recall and specificity on knowledge elements
        knowledge_mask = get_mask(X_val, y_val, maxes)
        target_0_knowledge = y_val[knowledge_mask & (y_val==0)]
        prediction_on_target_0 = prediction[knowledge_mask & (y_val==0)]>0.5
        target_1_knowledge = y_val[knowledge_mask & (y_val==1)]
        prediction_on_target_1 = prediction[knowledge_mask & (y_val==1)]>0.5
        

        all_score.append([
            accuracy_score(y_val, prediction>0.5), # accuracy
            balanced_accuracy_score(y_val, prediction>0.5), # balanced accuracy
            recall_score(y_val, prediction>0.5), # recall
            recall_score(y_val == 0, prediction<0.5), # recall on 0
            f1_score(y_val, prediction>0.5), # f1score
        ])

        ######
        # Evaluating model recall and specificity on knowledge elements
        if(len(target_0_knowledge) > 0 and len(prediction_on_target_0)>0):
            specificity_knowledge += accuracy_score(target_0_knowledge, prediction_on_target_0)
            sp_kn_count += 1

        if(len(target_1_knowledge) > 0 and len(prediction_on_target_1)>0):
            recall_knowledge += accuracy_score(target_1_knowledge, prediction_on_target_1)
            re_kn_count += 1
        ######

    all_score = np.sum(np.array(all_score), axis=0)
    all_score = all_score / n_splits

    specificity_knowledge /= sp_kn_count
    recall_knowledge /= re_kn_count

    all_score=np.append(all_score, specificity_knowledge)
    all_score=np.append(all_score, recall_knowledge)

    test_performances[current_test_name] = all_score
    


Using device: cpu
Epoch 0/100
 18/18 [==============================] - loss: 0.6983817219734192
Epoch 1/100
 18/18 [==============================] - loss: 0.6906030178070068
Epoch 2/100
 18/18 [==============================] - loss: 0.6855918765068054
Epoch 3/100
 18/18 [==============================] - loss: 0.6970923542976379
Epoch 4/100
 18/18 [==============================] - loss: 0.6868855953216553
Epoch 5/100
 18/18 [==============================] - loss: 0.6445118188858032
Epoch 6/100
 18/18 [==============================] - loss: 0.6600196361541748
Epoch 7/100
 18/18 [==============================] - loss: 0.6218133568763733
Epoch 8/100
 18/18 [==============================] - loss: 0.6056683063507089
Epoch 9/100
 18/18 [==============================] - loss: 0.6887819766998291
Epoch 10/100
 18/18 [==============================] - loss: 0.6378227472305298
Epoch 11/100
 18/18 [==============================] - loss: 0.6032403707504272
Epoch 12/100
 18/18 [===========

In [17]:
# Plotting performances table
table = pd.DataFrame(test_performances)

metrics = ["Accuracy", "Balanced", "Recall", "Specificity", "F1","knowledge_0", "knowledge_1"]

data = []
for model, values in test_performances.items():
    row = np.append(np.array([model]), values)
    data.append(row)

columns = ["Model"] + metrics
table = pd.DataFrame(data, columns=columns)
table


,Model,Accuracy,Balanced,Recall,Specificity,F1,knowledge_0,knowledge_1
0,LTN,0.76953125,0.7284030607372497,0.5961269560282988,0.8606791654462006,0.6425259138025096,0.9923076923076923,1.0
